In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformer_lens import HookedTransformer
from sae_lens import SAE
from tqdm.notebook import tqdm
from IPython.display import IFrame
from interpolated_ffn import ModelWithBilinearLayer, load_layer

device = 'cuda'
dtype = torch.bfloat16

Load the Gemma 2 and Gemma Scope weights from HuggingFace.

In [2]:
model_name = "gemma-2-2b"
model = HookedTransformer.from_pretrained_no_processing(model_name, device = device, dtype=dtype)

layer = 18

sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res-canonical",
    sae_id = f"layer_{layer}/width_16k/canonical",
    device = device
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 15.66 GiB of which 29.44 MiB is free. Process 1745774 has 8.28 GiB memory in use. Including non-PyTorch memory, this process has 6.14 GiB memory in use. Of the allocated memory 5.74 GiB is allocated by PyTorch, and 157.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model_bilinear = ModelWithBilinearLayer(model, layer)
load_layer(model_bilinear, f"bilinear_layer{layer}.pt")